In [ ]:
import math
import pandas as pd
%load_ext dotenv
%dotenv
import os
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
from email import encoders

import mimetypes
from os.path import basename
import smtplib
import re
import jinja2
import yaml
import logging
from logging.config import dictConfig

# with open("logconf.yml", 'r') as f:
#     log_config = yaml.safe_load(f.read())
# dictConfig(log_config)
logger = logging.\
    getLogger(__name__)

In [ ]:

mailserver = smtplib.SMTP('smtp.gmail.com',587)
mailserver.ehlo()
mailserver.starttls()
mailserver.login(os.environ.get('GMAIL_USER'), os.environ.get('GMAIL_PW'))
# mailserver = smtplib.SMTP('smtp.office365.com',25)
# mailserver.ehlo()
# mailserver.starttls()
# print(os.environ.get('UOB_USER'))
# mailserver.login(os.environ.get('UOB_USER'), os.environ.get('UOB_PW'))


jinja_environment = jinja2.Environment(loader=jinja2.FileSystemLoader(os.path.abspath(os.getcwd()) + '/templates'))
# template = jinja_environment.get_template('unsuccessful_project_email.html')
template = jinja_environment.get_template('portfolio_feedback_email.html')


In [ ]:
import pandas as pd

"""
Script to prepare feedback files and BB upload summary sheets.

A bunch of files are required with student and group info.

Because xl marksheets have names not identical to groups, the first step is to compute a map of 
student <> group marksheet filename

https://www.ole.bris.ac.uk/bbcswebdav/institution/TEL/EMA%20guidance/Published%20guides/B1.4%20Marking%20and%20Moderation.pdf   
"""

"the BB assessment download file"
assessment_df = pd.read_excel(
    'Blackboard marking file .xls',
    skiprows=3)

"student_list.xlsx -- direct download from evision MyStudents "
df_students = pd.read_excel(
    'student_list.xlsx',
    sheet_name='Sheet1')

"combined marking sheet for all portfolios. created by `merge_xls.ipynb`"
df_marks = pd.read_excel(
    'marks.xlsx',
    sheet_name='marks_short')


student_files = {}
# get xl file name for each student who submitted
for student in assessment_df['Username'].values:
    
    group_name = 'XXX'
    student_files[student] = group_name


import glob

# load all files
files = glob.glob(
    'Marksheets/*.xlsx', )

markssheet_paths = {}
for f in files:
    import os

    group = os.path.basename(f).split('.')[0]
    markssheet_paths[group] = f

from shutil import copyfile

In [ ]:
def add_attachments(path):
            
    ctype, encoding = mimetypes.guess_type(path)
    if ctype is None or encoding is not None:
        # No guess could be made, or the file is encoded (compressed), so
        # use a generic bag-of-bits type.
        ctype = 'application/octet-stream'
    maintype, subtype = ctype.split('/', 1)
    
    with open(path, 'rb') as fp:
        part = MIMEBase(maintype, subtype)
        part.set_payload(fp.read())
        # Encode the payload using Base64
        encoders.encode_base64(part)

    part.add_header('Content-Disposition', 'attachment', filename=path)
    msg.attach(part)
    
        

"""
for each submission on blackboard
1. find the student
2. find the group
3. get the mark for the group and write it to the relevant excel file
4. copy the excel file into a returns folder
"""
for i, row in assessment_df.iterrows():
    username_ = row['Username']
    logfile.write(username_)
    group_name = student_files[username_]
    logfile.write(group_name)

    mark = df_marks.loc[df_marks['group'] == group_name]['mark'].values[0]
    logfile.write(mark)
    ifor_val = mark
    assessment_df.at[i, 'mark'] = ifor_val

    student_number = df_students.loc[df_students['COMMON USERNAME'] == username_]['STUDENT NUMBER'].values[0]
    file_path = markssheet_paths[group_name]
    student_identifier = username_
    


    student_emails = [username_+'@my.bristol.ac.uk']
    item = {'projectName':group_name,                                              
            'studentEmails': student_emails,            
    }
    
    context = {'projectName': item['projectName'], 'mark':mark}

    body = template.render(context)

    msg = MIMEMultipart()
    send_from = 'XXX'
    msg['From'] = send_from
    send_to = student_emails
    msg['To'] = COMMASPACE.join(send_to)
    cc_adr = 'XXX'

    msg['CC'] = cc_adr
    msg['Date'] = formatdate(localtime=True)
    msg['Subject'] = f'Feedback {group_name}'
    
    msg.attach(MIMEText(body, 'html'))
    
    
    add_attachments(file_path)

    with open('log.txt', 'a+') as logfile:
        logfile.write(f'sent email for project {group_name}\n')
        logfile.write(f'\temails: {send_to}\n')
        
    
    mailserver.sendmail(send_from, send_to + [cc_adr] + student_emails, msg.as_string())    

In [ ]:
mailserver.quit()

#